In [2]:
from math import gcd

'''
http://robin.pollak.io/wizard_factoring.pdf
'''

pq1 = (4715832397,5476546811)
n1 = pq1[0]*pq1[1]

pq2 = (47217592089965824021278357521901144689506741271419, 19244123869039008990587632107835659152065053045809)
n2  = pq2[0] * pq2[1] 

pq3 = (9748732579,6892276889)
n3 = pq3[0] * pq3[1] 

pq1 = (471841,270689)
n1 = pq1[0]*pq1[1]
n4 = 1035896947 *3500688283
n = 11321*22943
#n = 94343*23143
n = 24430436836352483797063096106414774550335403158860524990373856944254297586018983298705492233654830523199692736514219239319146510300625339876324647894404893431924815784960576423322270642064447439532880084271906506181596791330255472317736274838975886982878636272375371266076163692482358755032762905350584467818322896256697824686362875462689754439034305568485764372282041358336076968265549938271297558180529807346415680592340188076635410423523198461463168140415659131710799290510682225326203262832075315314659506065014839527357257670962979561415126272979988642044180466409976304796211964778255235614585976098809592350027




#n =n4
a = 2
i = 2
d = 1

In [3]:
def solve_pollard(n):
    print("Refactoring N with Pollard P-1 Method:")
    a = 2
    B = 100000
    for i in range(2,B):
        a = pow(a,i,n)
        p = gcd(a-1,n)
        if p>1:
            if n%p ==0:
                q = n//p
                result = {}
                result['p'] = p
                result['q'] = q
                print("Pollar P-1 Method finished, N was refactored")
                return result
                break              
    print("Pollar P-1 Method finished, Failed")
    return False

In [4]:
if __name__ == "__main__":
    n = 1035896947 *3500688283
    print('This is to test Pollard P-1 Algorithm')
    print( 'n = ',n)
    pq = solve_pollard(n)
    print ("p = ", pq['p'])
    print ("q = ", pq['q'])

This is to test Pollard P-1 Algorithm
n =  3626352304758372001
Refactoring N with Pollard P-1 Method:
Pollar P-1 Method finished, N was refactored
p =  1035896947
q =  3500688283
